## Data Sources

TSA Claims https://www.dhs.gov/tsa-claims-data   
Airline coordinates http://openflights.org/data.html  
Airline passenger enplanements https://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/

# TSA Data

This is data published by the Department of Homeland Security of claims made against the Transportation Security Administration. It includes the airline of the passenger, airport, claim site, claim type, close amount, date of the incident and report, disposition, and type of claim and item. We are interested in data beginning in 2007 and ending in 2015. This report will outline the analysis procedure and explore the data to answer questions of interest.

We first begin by importing the requisite packages and libraries, and then reading in the data. The data are split over 4 excel files in both xlsx and xls formats. We will import them as pandas dataframes and append them together into one larger dataframe. Beforehand, the name of one column in one of the files was renamed to match with the others, though no other modifications to the raw data were made.

In [1]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import glob
import plotly.plotly as py
import plotly.graph_objs as go


In [2]:
directory = "data/"
data = pd.read_excel("data/claims-data-2015-as-of-feb-9-2016.xlsx")
for filename in glob.glob(directory + "*.xls"):
    data = data.append(pd.read_excel(filename))
data = data.iloc[:,[0,1,5,6,7,8,9,10,11]].reset_index()
del data['index']

Here are the first 5 rows of our dataframe. There are missing data present in some entries, which will be dealt with in the next step. Columns which were not required(such as airport name, which is redundant due to the airport code) were removed.

In [3]:
data.head()

,Airline Name,Airport Code,Claim Site,Claim Type,Close Amount,Date Received,Disposition,Incident Date,Item Category
0,Allegiant Air,ABE,Checkpoint,Property Damage,0,2015-05-28,Deny,2015-05-21 00:00:00,Personal Electronics
1,-,ABE,Checked Baggage,Property Damage,180,2015-08-21,Settle,2015-08-01 00:00:00,Sporting Equipment & Supplies
2,Allegiant Air,ABE,Checked Baggage,Passenger Property Loss,60,2015-10-07,Approve in Full,2015-10-02 00:00:00,Clothing
3,Allegiant Air,ABE,Checkpoint,Property Damage,-,2015-10-27,-,2015-10-19 00:00:00,Personal Accessories
4,American Airlines,ABI,Checked Baggage,Property Damage,0,2015-01-22,Deny,2014-12-23 00:00:00,Home Decor


We remove entries without an airline and without a disposition, and we will only focus on claims for property damage and passenger property loss, since they are the most common and most relevant to most passengers. Rows with NA values were removed as well. This does not imply that these data are not useful, and further analysis on missingness might yield insights. Our final dataset contains 82,589 claims.

In [4]:
data['Claim Type'].groupby(data['Claim Type']).count()

Claim Type
-                            282
Bus Terminal                   1
Complaint                     48
Compliment                     3
Employee Loss (MPCECA)        71
Motor Vehicle                307
Passenger Property Loss    67700
Passenger Theft               24
Personal Injury              972
Property Damage            37349
Wrongful Death                 4
Name: Claim Type, dtype: int64

In [5]:
data = data[data["Disposition"] != '-']
data = data[data["Airline Name"] != '-']
data = data[(data['Claim Type']  =="Property Damage" ) | (data['Claim Type']  =="Passenger Property Loss" )].dropna()

In [6]:
data.shape

(82589, 9)

In [7]:
data.head()

,Airline Name,Airport Code,Claim Site,Claim Type,Close Amount,Date Received,Disposition,Incident Date,Item Category
0,Allegiant Air,ABE,Checkpoint,Property Damage,0,2015-05-28,Deny,2015-05-21 00:00:00,Personal Electronics
2,Allegiant Air,ABE,Checked Baggage,Passenger Property Loss,60,2015-10-07,Approve in Full,2015-10-02 00:00:00,Clothing
4,American Airlines,ABI,Checked Baggage,Property Damage,0,2015-01-22,Deny,2014-12-23 00:00:00,Home Decor
5,American Airlines,ABI,Checked Baggage,Passenger Property Loss,200,2015-08-05,Approve in Full,2015-07-14 00:00:00,Baggage/Cases/Purses
6,American Airlines,ABI,Checked Baggage,Passenger Property Loss,0,2015-09-07,Deny,2015-08-06 00:00:00,Personal Electronics


## Analysis

### Which Airports Have the Most Claims?

To answer this question, we'll start by grouping the claims by airport. Airports such as JFK, LAX, amd MCO have the greatest numbers of claims, at over 3000.

In [8]:
airlinecounts = pd.DataFrame({'count' : data['Airport Code'].groupby(data['Airport Code']).size().sort_values()}).reset_index()

In [9]:
airlinecounts.tail()

,Airport Code,count
419,ATL,2873
420,EWR,2902
421,MCO,3319
422,LAX,3583
423,JFK,4828


This statistic does not account for the passenger throughput of the airports. One would expect that more passengers would yield more claims. Using enplanement data published by the FAA on commercial service airports, we can merge our airport data with the number of enplanements. The dataset includes other columns as well, but we will use the 2015 enplanements to gauge the volume of passengers. The Rank column is ordered by number of enplanements. 

For later plots, we will also introduce the coordinates of each airport to our data. These coordinates are published by openflights.org.

In [10]:
throughput = pd.read_excel("cy15-commercial-service-enplanements.xlsx")
airlines = airlinecounts.merge(throughput, on ="Airport Code")
airlines.iloc[:,[0,1,2,9]].head()

,Airport Code,count,Rank,CY 15 Enplanements
0,VDZ,1,370,14153
1,IWD,1,465,4697
2,IRK,1,449,5167
3,PPG,1,261,65860
4,FMN,1,519,3083


We can plot the number of enplanements vs counts. From both plots we see that there is a strong positive correlation. As predicted, as enplanements increases, so do claims.

In [11]:
fig = {
    'data': [
  		{
  			'x': airlines['CY 15 Enplanements'], 
        	'y': airlines['count'],
        	'text': airlines['Airport Code'], 
        	'mode': 'markers'
        }

    ],
    'layout': {
        'xaxis': {'title': 'Passengers in 2015' },
        'yaxis': {'title': 'Number of Claims'}
    }
}


py.iplot(fig, filename='basic-scatter')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~andrewchin/0 or inside your plot.ly account where it is named 'basic-scatter'


In [12]:
data['Claim Type'].groupby(data['Claim Type']).count()

Claim Type
Passenger Property Loss    53220
Property Damage            29369
Name: Claim Type, dtype: int64

In [24]:
coords = pd.read_csv("airports.dat", header = None).iloc[:,[4,6,7]]
coords.columns = ["Airport Code", "Lat", "Long"]
coords.head()

,Airport Code,Lat,Long
0,GKA,-6.081690,145.391998
1,MAG,-5.207080,145.789001
2,HGU,-5.826790,144.296005
3,LAE,-6.569803,146.725977
4,POM,-9.443380,147.220001
